In [1]:
import os

import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from IPython.display import display
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GroupKFold, KFold, train_test_split
from sklearn.preprocessing import (KBinsDiscretizer, OrdinalEncoder,
                                   RobustScaler, normalize)
from sklearn import model_selection


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
df = pd.read_csv("/home/koga/workspace/dataset/kaggle_ventilator/train_5folds.csv")
test_df = pd.read_csv("/home/koga/workspace/dataset/kaggle_ventilator/test.csv")

In [3]:
def add_features(df, transform="robust"):
    df["u_in_log1p"] = np.log1p(df["u_in"])
    df["u_in_power"] = np.power(df["u_in"], 2)
    
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()

    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
    df = df.drop(['count','one'], axis=1)
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    
    df['u_in_lag_back10'] = df.groupby('breath_id')['u_in'].shift(-10)
    df['u_out_lag_back10'] = df.groupby('breath_id')['u_out'].shift(-10)

    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']

    r_map = {5: 0, 20: 1, 50: 2}
    c_map = {10: 0, 20: 1, 50: 2}
    rc_map = {'20_50': 0, '20_20': 1, '50_20': 2, '50_50': 3, '5_50': 4, '5_20': 5, '50_10': 6, '20_10': 7, '5_10': 8}
    df['RC'] = df['R'].astype(str) + '_' + df['C'].astype(str)
    
    df['R'] = df['R'].map(r_map)
    df['C'] = df['C'].map(c_map)
    
    df['RC'] = df['RC'].map(rc_map)
    return df


In [4]:
train_df = add_features(df)
test_df = add_features(test_df)

In [5]:
sorted(train_df.columns)

['C',
 'R',
 'RC',
 'area',
 'breath_id',
 'breath_id__u_in__diffmax',
 'breath_id__u_in__diffmean',
 'breath_id__u_in__max',
 'breath_id__u_out__max',
 'cross',
 'cross2',
 'delta',
 'id',
 'kfold',
 'pressure',
 'time_delta',
 'time_step',
 'u_in',
 'u_in_cummean',
 'u_in_cumsum',
 'u_in_diff1',
 'u_in_diff2',
 'u_in_diff3',
 'u_in_diff4',
 'u_in_lag1',
 'u_in_lag2',
 'u_in_lag3',
 'u_in_lag4',
 'u_in_lag_back1',
 'u_in_lag_back10',
 'u_in_lag_back2',
 'u_in_lag_back3',
 'u_in_lag_back4',
 'u_in_log1p',
 'u_in_power',
 'u_out',
 'u_out_diff1',
 'u_out_diff2',
 'u_out_diff3',
 'u_out_diff4',
 'u_out_lag1',
 'u_out_lag2',
 'u_out_lag3',
 'u_out_lag4',
 'u_out_lag_back1',
 'u_out_lag_back10',
 'u_out_lag_back2',
 'u_out_lag_back3',
 'u_out_lag_back4']

In [6]:
norm_features = [
    'area',
    'breath_id__u_in__diffmax',
    'breath_id__u_in__diffmean',
    'breath_id__u_in__max',
    'cross',
    'cross2',
    'delta',
    'time_delta',
    'time_step',
    'u_in',
    'u_in_cummean',
    'u_in_cumsum',
    'u_in_diff1',
    'u_in_diff2',
    'u_in_diff3',
    'u_in_diff4',
    'u_in_lag1',
    'u_in_lag2',
    'u_in_lag3',
    'u_in_lag4',
    'u_in_lag_back1',
    'u_in_lag_back10',
    'u_in_lag_back2',
    'u_in_lag_back3',
    'u_in_lag_back4',
    'u_in_log1p',
    'u_in_power',
 ]

In [7]:
RS = RobustScaler()
train_df[norm_features] = RS.fit_transform(train_df[norm_features])
test_df[norm_features] = RS.fit_transform(test_df[norm_features])

In [8]:
train_df["u_in_round2"] = np.round(train_df["u_in"], 2)
test_df["u_in_round2"] = np.round(test_df["u_in"], 2)

In [10]:
train_df["kfold"] = -1
y = train_df["RC"].values

# kf = model_selection.GroupKFold(n_splits=5)
kf = model_selection.StratifiedGroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y, groups=train_df.breath_id.values)):
    print(f)
    train_df.loc[v_, "kfold"] = f

0
1
2
3
4


In [13]:
train_df.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_power,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag_back10,u_out_lag_back10,breath_id__u_in__max,breath_id__u_out__max,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,RC,u_in_round2
0,1,1,1,2,-0.989052,-0.937384,0,5.837492,0,-1.100643,-0.779094,-15.475938,-0.860663,-0.734997,0.0,-0.665541,-0.725228,-0.648438,-0.864121,0.0,2.890207,0.0,-0.842527,0.0,3.664026,0.0,-0.819258,0.0,3.743915,0.0,-0.794152,0.0,4.277824,0.0,4.778007,0.0,0.112208,1,0.471409,0.0,0.217993,0.0,0.326941,1.273864,0.121028,0.0,0.068315,0.0,0,-0.94
1,2,1,1,2,-0.963608,3.049278,0,5.907794,0,0.878825,12.910990,0.103311,2.863270,-0.683536,0.0,-0.665541,-0.676829,0.162935,-0.847360,0.0,3.739712,0.0,-0.842527,0.0,3.724218,0.0,-0.819258,0.0,4.256189,0.0,-0.794152,0.0,4.661047,0.0,4.756926,0.0,0.112208,1,113.467773,0.0,58.762670,0.0,-0.178436,-1.450205,38.091005,0.0,26.306250,0.0,0,3.05
2,3,1,1,2,-0.938006,3.948195,0,7.876254,0,1.011274,19.746583,0.200442,3.727568,-0.620131,0.0,-0.665541,-0.617568,0.555359,2.833140,0.0,3.801381,0.0,-0.825784,0.0,4.236036,0.0,-0.819258,0.0,4.639136,0.0,-0.794152,0.0,4.634403,0.0,5.006451,0.0,0.112208,1,25.549576,0.0,71.696779,0.0,-0.292389,-2.064430,46.652513,0.0,32.222406,0.0,0,3.95
3,4,1,1,2,-0.912278,4.013452,0,11.742872,0,1.019963,20.296520,0.277152,3.811377,-0.555568,0.0,-0.665541,-0.557517,0.758212,3.663022,0.0,4.325760,0.0,2.850764,0.0,4.618643,0.0,-0.802501,0.0,4.612511,0.0,-0.794152,0.0,4.570046,0.0,4.762421,0.0,0.112208,1,1.812565,0.0,14.110409,0.0,-0.300662,-2.109020,47.101127,0.0,32.651890,0.0,0,4.01
4,5,1,1,2,-0.886409,4.568332,0,12.234987,0,1.089713,25.266363,0.363024,4.361409,-0.483405,0.0,-0.665541,-0.490761,0.925096,3.723268,0.0,4.717757,0.0,3.679756,0.0,4.592042,0.0,2.877119,0.0,4.548200,0.0,-0.777379,0.0,4.782785,0.0,4.528033,0.0,0.112208,1,15.753730,0.0,9.058184,0.0,-0.371002,-2.488167,14.415967,0.0,36.184309,0.0,0,4.57


In [14]:
train_df["kfold"].value_counts()

0    1207280
1    1207200
2    1207200
3    1207200
4    1207120
Name: kfold, dtype: int64

In [12]:
train_df.to_csv("../../dataset/kaggle_ventilator/train_5folds_nb11_robust.csv", index=False)
test_df.to_csv("../../dataset/kaggle_ventilator/test_nb11_robust.csv", index=False)

In [109]:
train_df = pd.read_csv("../../dataset/kaggle_ventilator/train_5folds_nb11_robust.csv")
test_df = pd.read_csv("../../dataset/kaggle_ventilator/test_nb11_robust.csv")

In [15]:
t_df = train_df[train_df["kfold"] != 0]
v_df = train_df[train_df["kfold"] == 0]

In [16]:
import category_encoders as ce


def add_target_encoding(tr_df, val_df, n_bins=500,
                        strategy='uniform', feature='u_in',
                        smoothing=1, skip_test=True):
    if smoothing == 1:
        sm = ''
    else:
        sm = '_' + str(smoothing)
        
    kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)
    tr_df[f'{feature}_{n_bins}bins{sm}'] = kbd.fit_transform(tr_df[feature].values.reshape(-1, 1))
    val_df[f'{feature}_{n_bins}bins{sm}'] = kbd.transform(val_df[feature].values.reshape(-1, 1))

    tr_df[f'{feature}_{n_bins}bins{sm}']  = tr_df[f'{feature}_{n_bins}bins{sm}'].astype('category')
    val_df[f'{feature}_{n_bins}bins{sm}']  = val_df[f'{feature}_{n_bins}bins{sm}'].astype('category')

    feature_name = f'target_encode_{feature}_{n_bins}bins{sm}'
    te = ce.target_encoder.TargetEncoder(verbose=1, smoothing=smoothing)
    tr_df[feature_name] = te.fit_transform(tr_df[f'{feature}_{n_bins}bins{sm}'], tr_df['pressure']) 
    val_df[feature_name] = te.transform(val_df[f'{feature}_{n_bins}bins{sm}']) 


    te_col_name = f'target_encode_{feature}_{n_bins}bins{sm}'

    # if te_col_name not in features:
    #     features.append(te_col_name)

    return tr_df, val_df, feature_name

In [17]:
t_df, v_df, feature_name = add_target_encoding(t_df, v_df, n_bins=300, feature="u_in", smoothing=1)
t_df, v_df, feature_name = add_target_encoding(t_df, v_df, n_bins=600, feature="u_in", smoothing=1)
t_df, v_df, feature_name = add_target_encoding(t_df, v_df, n_bins=1000, feature="u_in", smoothing=1)
t_df, v_df, feature_name = add_target_encoding(t_df, v_df, n_bins=3, feature="u_in", smoothing=1)
t_df, v_df, feature_name = add_target_encoding(t_df, v_df, n_bins=500, feature="u_in_round2", smoothing=10)


/home/koga/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/koga/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/koga/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [22]:
t_df["target_encode_u_in_1000bins"].value_counts()

12.082295    1161439
6.354665      612710
6.651075      245441
6.891830      151308
7.108406      103128
              ...   
16.065706         47
16.821438         46
15.455131         46
17.494568         43
16.576981         42
Name: target_encode_u_in_1000bins, Length: 999, dtype: int64

In [18]:
t_df.columns

Index(['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure', 'kfold', 'u_in_log1p', 'u_in_power', 'time_delta', 'delta', 'area', 'cross', 'cross2', 'u_in_cumsum', 'u_in_cummean', 'u_in_lag1', 'u_out_lag1', 'u_in_lag_back1', 'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2', 'u_in_lag_back2', 'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3', 'u_in_lag_back3', 'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4', 'u_in_lag_back4', 'u_out_lag_back4', 'u_in_lag_back10', 'u_out_lag_back10', 'breath_id__u_in__max', 'breath_id__u_out__max', 'u_in_diff1', 'u_out_diff1', 'u_in_diff2', 'u_out_diff2', 'breath_id__u_in__diffmax', 'breath_id__u_in__diffmean', 'u_in_diff3', 'u_out_diff3', 'u_in_diff4', 'u_out_diff4', 'RC', 'u_in_round2', 'u_in_300bins', 'target_encode_u_in_300bins', 'u_in_600bins', 'target_encode_u_in_600bins', 'u_in_1000bins', 'target_encode_u_in_1000bins', 'u_in_3bins', 'target_encode_u_in_3bins', 'u_in_round2_500bins_10', 'target_encode_u_in_round2_500bins_10'], dtype='object')

In [110]:
train_df.head()

,level_0,index,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_power,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag_back10,u_out_lag_back10,breath_id__u_in__max,breath_id__u_out__max,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,RC,u_in_round2
0,80,160,161,3,2,1,-0.989052,-0.955539,0,5.064168,0,-1.155575,-0.779375,-15.475938,-0.860663,-0.734997,0.0,-0.665541,-0.725447,-0.655828,-0.864121,0.0,0.585230,0.0,-0.842527,0.0,1.821240,0.0,-0.819258,0.0,2.598003,0.0,-0.794152,0.0,2.868552,0.0,1.594169,0.0,-0.12784,1,-0.045513,0.0,-0.048610,0.0,0.054206,0.588359,-0.051881,0.0,-0.051169,0.0,2,-0.96
1,81,161,162,3,2,1,-0.964885,0.610230,0,5.064168,0,0.287377,1.313315,-0.678366,0.522237,-0.715887,0.0,-0.665541,-0.706525,-0.337160,-0.864121,0.0,1.851701,0.0,-0.842527,0.0,2.579324,0.0,-0.819258,0.0,2.847931,0.0,-0.794152,0.0,2.680101,0.0,1.522755,0.0,-0.12784,1,44.537027,0.0,22.944914,0.0,-0.144282,-0.481524,14.860893,0.0,10.253826,0.0,2,0.61
2,82,162,163,3,2,1,-0.940699,1.950366,0,6.751420,0,0.671960,6.428588,-0.666998,1.707828,-0.680393,0.0,-0.665541,-0.671407,-0.049105,0.581402,0.0,2.628390,0.0,-0.842527,0.0,2.829030,0.0,-0.819258,0.0,2.659615,0.0,-0.794152,0.0,2.264565,0.0,1.347486,0.0,-0.12784,1,38.112549,0.0,42.625001,0.0,-0.314167,-1.397233,27.624692,0.0,19.073840,0.0,2,1.95
3,83,163,164,3,2,1,-0.916549,2.772232,0,12.305289,0,0.832254,11.082357,-0.689404,2.429279,-0.634929,0.0,-0.665541,-0.626357,0.178556,1.818621,0.0,2.884224,0.0,0.601444,0.0,2.640881,0.0,-0.819258,0.0,2.244379,0.0,-0.794152,0.0,1.920148,0.0,1.290870,0.0,-0.12784,1,23.355685,0.0,31.700687,0.0,-0.418352,-1.958810,35.452348,0.0,24.482891,0.0,2,2.77
4,84,164,165,3,2,1,-0.892371,3.042947,0,18.702784,0,0.877795,12.867739,-0.671523,2.672466,-0.586105,0.0,-0.665541,-0.578035,0.337191,2.577370,0.0,2.691458,0.0,1.837334,0.0,2.226014,0.0,0.625919,0.0,1.900210,0.0,-0.794152,0.0,1.657469,0.0,1.330173,0.0,-0.12784,1,7.662626,0.0,15.996086,0.0,-0.452670,-2.143788,23.117931,0.0,26.264582,0.0,2,3.04


In [111]:
d = pd.read_csv("../../dataset/kaggle_ventilator/train_5folds_nb10_robust.csv")

In [112]:
d.head()

,id,breath_id,time_step,u_in,u_out,pressure,kfold,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,C_cate,R_cate,RC_sum,RC_dot
0,1,1,0.000000,0.083334,0,5.837492,4,0.000000,0.000000,0.000000,0.0,0.0,0.083334,0.083334,0.000000,0.0,18.383041,0.0,0.000000,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,28.313036,28.229702,10.062673,2,1,6,7
1,2,1,0.033652,18.383041,0,5.907794,4,0.033652,0.618632,0.618632,0.0,0.0,18.466375,9.233188,0.083334,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,28.313036,9.929994,-8.237035,2,1,6,7
2,3,1,0.067514,22.509278,0,7.876254,4,0.033862,0.762212,1.380843,0.0,0.0,40.975653,13.658551,18.383041,0.0,22.808822,0.0,0.083334,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,0.000000,0.0,27.127486,0.0,28.313036,5.803758,-12.363271,2,1,6,7
3,4,1,0.101542,22.808822,0,11.742872,4,0.034028,0.776134,2.156978,0.0,0.0,63.784476,15.946119,22.509278,0.0,25.355850,0.0,18.383041,0.0,27.259866,0.0,0.083334,0.0,27.127486,0.0,0.000000,0.0,26.807732,0.0,28.313036,5.504214,-12.662816,2,1,6,7
4,5,1,0.135756,25.355850,0,12.234987,4,0.034213,0.867507,3.024485,0.0,0.0,89.140326,17.828065,22.808822,0.0,27.259866,0.0,22.509278,0.0,27.127486,0.0,18.383041,0.0,26.807732,0.0,0.083334,0.0,27.864715,0.0,28.313036,2.957185,-15.209844,2,1,6,7


In [113]:
d["kfold"] = -1
y = d["C_cate"].values

# kf = model_selection.GroupKFold(n_splits=5)
kf = model_selection.StratifiedGroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=d, y=y, groups=d.breath_id.values)):
    print(f)
    d.loc[v_, "kfold"] = f

0
1
2
3
4


In [114]:
d.head()

,id,breath_id,time_step,u_in,u_out,pressure,kfold,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,C_cate,R_cate,RC_sum,RC_dot
0,1,1,0.000000,0.083334,0,5.837492,0,0.000000,0.000000,0.000000,0.0,0.0,0.083334,0.083334,0.000000,0.0,18.383041,0.0,0.000000,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,28.313036,28.229702,10.062673,2,1,6,7
1,2,1,0.033652,18.383041,0,5.907794,0,0.033652,0.618632,0.618632,0.0,0.0,18.466375,9.233188,0.083334,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,28.313036,9.929994,-8.237035,2,1,6,7
2,3,1,0.067514,22.509278,0,7.876254,0,0.033862,0.762212,1.380843,0.0,0.0,40.975653,13.658551,18.383041,0.0,22.808822,0.0,0.083334,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,0.000000,0.0,27.127486,0.0,28.313036,5.803758,-12.363271,2,1,6,7
3,4,1,0.101542,22.808822,0,11.742872,0,0.034028,0.776134,2.156978,0.0,0.0,63.784476,15.946119,22.509278,0.0,25.355850,0.0,18.383041,0.0,27.259866,0.0,0.083334,0.0,27.127486,0.0,0.000000,0.0,26.807732,0.0,28.313036,5.504214,-12.662816,2,1,6,7
4,5,1,0.135756,25.355850,0,12.234987,0,0.034213,0.867507,3.024485,0.0,0.0,89.140326,17.828065,22.808822,0.0,27.259866,0.0,22.509278,0.0,27.127486,0.0,18.383041,0.0,26.807732,0.0,0.083334,0.0,27.864715,0.0,28.313036,2.957185,-15.209844,2,1,6,7
